## 색맹테스트 봇
* http://zzzscore.com/color/

* 웹 페이지 코드 분석

* 버튼이 4->9->16->... 이런식으로 늘어남


* (1,1) 위치 div의 xpath : //*[@id="grid"]/div[1]
* (1,2) 위치 div의 xpath : //*[@id="grid"]/div[2]
* (2,1) 위치 div의 xpath : //*[@id="grid"]/div[3]


* 이런식이지만 div[*] 쓰려면 내부요소가 있는 경우에만 탐색 가능


* 이번에는 div와div 태그 사이에 아무런 요소가 없다. 이런 경우에는 //*[@id="grid"]/div 이런식으로 탐색 

In [1]:
from selenium import webdriver
from pprint import pprint
import time

driver = webdriver.Chrome("chromedriver")
driver.get('http://zzzscore.com/color')
driver.implicitly_wait(300)

btns = driver.find_elements_by_xpath('//*[@id="grid"]/div')
print(len(btns))

230


* 정답 탐색하기


* 1. 각 div의 rgba값 추출
* 2. 추출한 rgba 중 값 하나 다른곳이 정답

* 디자인 정보(css) 추출


* selenium에서 추출한 요소는 value_of_css_property로 해당 요소의 css속성을 확인할 수 있습니다. css란 html 요소의 디자인 속성 정보입니다.


* 모든 버튼에 대해 배경색상 정보를 추출하여 리스트에 저장


In [2]:
btns_rgba = [btn.value_of_css_property('background-color') for btn in btns]
pprint(btns_rgba)

['rgba(163, 180, 195, 1)',
 'rgba(163, 180, 195, 1)',
 'rgba(130, 147, 228, 1)',
 'rgba(163, 180, 195, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 

* 정답 찾기


* btns_rgba 값 찾기 - >  collections모듈의 Counter함수를 이용, Counter함수는 값(value)과 개수를 딕셔너리(dict)로 반환해줍니다.


* 반환된 딕셔너리에서 value가 1인 것이 정답의 배경색 정보이다.

In [3]:
from collections import Counter

result = Counter(btns_rgba)
pprint(result)   # 여기서 나온 value가 1인게 정답이다.

# value가 1인 것을 탐색
for key, value in result.items():
    if value == 1:
        answer = key
        break

else:
    answer = None
    print("정답을 찾을 수 없습니다.")

Counter({'rgba(255, 0, 0, 1)': 226,
         'rgba(163, 180, 195, 1)': 3,
         'rgba(130, 147, 228, 1)': 1})


* 정답 누르기


* 버튼 요소를 저장한 btns에 배경색 정보를 순서대로 추출한 것이 btns_rgba 임으로 인덱스가 일치


* 1. answer에 저장된 색이 어느 위치(인덱스)에 있는지 btns_rgba에서 찾고
* 2. 그 인덱스로 btns에 저장된 요소를 조회하여 클릭을 한다.

In [4]:
# 정답 클릭
# 1. btns_rgba에서 인덱스 값을 구하고
# 2. 그 인덱스 값으로 btns 인덱스에 접근, 클릭
if answer:
    index = btns_rgba.index(answer)
    btns[index].click()

* 제한 시간동안 수행


* 반복하는 함수를 만들어서 60초동안 이를 계속 함수 호출이 되도록 만듬


* time.time()은 UTC기준부터 현재까지의 시간을 초로 반환해줍니다. 이를 이용해 60초가 지났는지 확인


In [7]:
from selenium import webdriver
from collections import Counter
from pprint import pprint
import time

driver = webdriver.Chrome('chromedriver')
driver.get('http://zzzscore.com/color/')
driver.implicitly_wait(300)

btns = driver.find_elements_by_xpath('//*[@id="grid"]/div')

# 반복되는 함수 생성
def analysis():
    # btns = driver.find_elements_by_xpath('//*[@id="grid"]/div')
    # print(len(btns))
    
    btns_rgba = [btn.value_of_css_property('background-color') for btn in btns]
    # pprint(btns_rgba)
    
    result = Counter(btns_rgba)
    # pprint(result)  # 여기서 value가 1인게 정답
    
    # value가 1인 것을 탐색
    for key, value in result.items():
        if value == 1:
            answer = key
            break
    
    else:
        answer = None
        print("정답을 찾을 수 없습니다.")
        
    # 정답 클릭
    # 1. btns_rgba에서 인덱스 값을 구하고
    # 2. 그 인덱스 값으로 btns 인덱스에 접근, 클릭
    if answer:
        index = btns_rgba.index(answer)
        btns[index].click()
        
if __name__=="__main__":
    start = time.time()
    while time.time() - start <= 60:
        analysis()

## 더 간편한 방법으로 하기


* 이 웹페이지의 소스코드를 보면 정답 버튼은 class="main"이라는 속성값을 가지고 있다.


* 그래서 css정보를 따로 추출하지 않아도 금방 할수 있음

In [8]:
from selenium import webdriver
from pprint import pprint
import time

driver = webdriver.Chrome('chromedriver')
driver.get('http://zzzscore.com/color/')
driver.implicitly_wait(300)

#시작 시각
start = time.time()

while time.time() - start <= 60:
    try:
        btn = driver.find_element_by_class_name("main")
        btn.click()
    except:
        pass